In [4]:
import requests
import json
import getpass
import pandas as pd
from pandas.io.json import json_normalize
import time

In [5]:
bearer_token = getpass.getpass()

In [6]:
headers = {
    'Authorization':'Bearer ' + bearer_token
}

In [7]:
midterm_2018_data = pd.read_json('../datasets/midterm-2018_processed_user_objects.json')

In [8]:
start = 4
end = 11
chunks = [midterm_2018_data['screen_name'][i:i + 300] for i in range(0, len(midterm_2018_data['screen_name']), 300)][start:end]
for num, chunk in enumerate(chunks, start=start):
    data = dict()
    for username in chunk:
        url = f'https://api.twitter.com/2/tweets/search/recent?query=from:{username}&tweet.fields=attachments,author_id,context_annotations,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang,public_metrics,possibly_sensitive,referenced_tweets,source,text,withheld'
        response = requests.request('GET',url,headers=headers)
        tweets = response.json()
        del tweets['meta']
        data[username] = tweets
    data2 = list()
    for user in data:
        data[user]['screen_name'] = user
        if('data' in data[user]):
            data[user]['recent_tweets'] = len(data[user]['data'])
        else:
            data[user]['recent_tweets'] = 0
        data2.append(data[user])
    with open(f'../tweets/midterm_2018_{num}.json','w') as outfile:
        json.dump(data2, outfile)
    time.sleep(60*15)